In [ ]:
# The preliminary notebook for testing/developing wrsamp. Will delete before merging. 

wrsamp calls wrheader with sig. wrheader calls setfields, and checkfields. 


setfields() - fills in fields of a wfdbrecord object. Fields that it can fill in include essential fields, and dependencies.

It will NOT overwrite any user defined fields, aka fields != None. 





Fields that depend on other fields:
    - 

Perhaps easiest solution is to specify a default order of filling in fields. Need to separate by record and signal? Nope, just put the record ones before the signal ones!!!!

In [2]:
# pseudocode.
# setfield and checkfield should NOT be visible to the user. Fuck them, don't want them shitting it up.
# It makes the code 100x harder to write. 
# We don't even need to remove foreign fields it seems. Who gives a shit? We don't even use them anyway. 

def wrsamp(record): # This belongs in signals.py
    
    record.checkfields([signals, physical])
    wrheader(record)
    wrdat(record)

    
def wrheader(record):
    
    writefields = record.getwritefields() # Get all the fields needed to write the record
    record.checkfields(writefields, setmissing = 1, fieldcheckorder = singlefieldspeclist) # 
    record.wrheaderfile()
    
# Get the list of required fields needed to write a wfdb record. 
# Returns the default required fields, the user defined fields, and their dependencies.
# Method of WFDBbaserecord
def getwritefields(self, fieldspeclist):

    # Record specification fields
    writefields=getwriterecfields(self, reversed(list(fieldspeclist[1].items())))

    # Single-segment record
    if fieldspeclist[0] == signalspecs:

        checkfield(self, 'nsig')

        if self.nsig>0:
            # signals is not always required because this method is called via wrheader. 
            # Enforce the presence of signals at the start of rdsamp to ensure it gets added here. 
            if self.signals!=None:
                writefields.append('signals')

            writefields=writefields+getreqsubset(self, reversed(list(fieldspeclist[2].items())))

    # Multi-segment record
    else:
        # Segment specification fields and segments
        writefields=writefields+['segments', 'seglen', 'segname']

    # Comments
    if self.comments !=None:
        reqfields.append('comments')
    return reqfields


# Get the record specification fields needed to write a wfdb header. Helper to getwritefields.
# Method of WFDBbaserecord
def getwriterecfields(self, fieldspecs):
    reqfields=[]
    for f in fieldspecs:
        if f in reqfields:
            continue
        # If the field is default required or has been defined by the user...
        if f[1].write_req or self.f!=None:
            rf=f
            # Add the field and its recurrent dependencies
            while rf!=None:
                reqfields.append(rf)
                rf=fieldspeclist[1][rf].dependency

    return reqfields    

# Check each field in the list of fields. If setmissing is true, try to set the field if it == None. 
# The checking order is specified by fieldcheckorder. Hopefully via the checking order, there will be no 
# Fieldcheckorder starts from record line and moves onto signal/segment line.
# This will belong to the base class. 
def checkfields(self, fieldstocheck, setmissing=0, fieldcheckorder = []):
    # Check fields in order because the validity of some may be dependent on others. 
    # In addition, the setvalue of some may depend on 
    for f in fieldcheckorder:
        if f in fieldstocheck:
            checkfield(self, f, setmissing)
                
                
# Check whether a field is valid. If the 'setfield' option is 1, the function will try to set a default if possible. 
# Checkfield calls setfield when setmissing == 1, and setfield may call checkfield (on another field) with setmissing = 0.
def checkfield(self, field, setmissing=0):
    
    errormsg = None
    
    # Try to set the value of the missing field if specified. 
    # Note, this may do nothing. 
    if setvalue and self.field==None:
        setfield(self, field)
        
    # If the field is still missing, trigger an error
    if self.field == None:
        sys.exit("Missing field required: "+field)
    
    if field==a:
            
    elif field==b:
           
        checkfield(e)
        
            
    elif field==b:    
    elif field==b:
    
    return errormsg
# The reason why this checkfield method returns an error message rather than directly triggering an error is because
# checkfield is called by checkfields on fields we need to have, but it is also called by setfield to check other
# fields needed to set default values. So in the first case, we will exit using the error message. In the second case
# we will ignore the error message and continue without being able to set the field. 


    
# Set the field if possible. Otherwise do nothing and return (no error message).
# This method WILL overwrite fields already set. Key is to not call it on fields already set to avoid this. 
# Although for gain and baseline when physical == 1, it will overwrite anyway. 
# Not all fields have defaults, and some fields' defaults may rely on other fields being present and valid. 
def setfield(self, field):
    
    # Record specification fields
    if field == 'nsig':
        if not checkfield(self, 'signals', 0):
            self.nsig = np.shape(self.signals)[1]
    elif field == 'siglen':
        if not checkfield(self, 'signals', 0):
            self.nsig = np.shape(self.signals)[1]
    elif field == 'basetime':
        self.basetime = '00:00:00'
        
    # Signal specification fields    
    # This method will only suggest single dat records
    elif field == 'filename':
        self.filename = self.nsig*[self.recordname+'.dat']
    elif field == 'fmt':
        if not checkfield(self, 'signals', 0):
            res = estres(self.signals)
            self.fmt = self.nsig*[wfdbfmt(max(res))]
    # adcgain, and baseline are related and depend on fmt.  
    # Do not infer a gain or baseline if the signal is already digital.
    elif field == 'adcgain':
        if self.physical == 1:
            # Calculate and set the gain and baseline.  
            if not checkfield(self, 'signals', 0):
                [gain, baseline] = adc(self.signals)
                self.adcgain = gain
                self.baseline = baseline
    elif field == 'baseline':
        if self.physical == 1:
            # Calculate and set the gain and baseline.  
            if not checkfield(self, 'signals', 0):
                [gain, baseline] = adc(self.signals)
                self.adcgain = gain
                self.baseline = baseline
    elif field == 'adcres':
        if not checkfield(self, 'fmt', 0):
            self.adcres=wfdbfmtres(self.fmt)
    elif field == 'adczero':
        if not checkfield(self, 'nsig', 0):
            self.adczero = self.nsig*[0]
    elif field == 'initvalue':   
        if not checkfield(self, 'signals', 0):
            self.initvalue = self.signals[0,:]
    elif field == 'checksum':
        if not checkfield(self, 'signals', 0):
            self.initvalue = calc_checksum(self.checksum)
    elif field == 'blocksize':
        self.blocksize = 0
        
    
    # To do for multirecord (or maybe it's better not in this function): elif field == 'seglen':
 

# Estimate the resolution of each signal in a multi-channel signal in bits. Maximum of 32 bits. 
reslevels = np.power(2, np.arange(0,33))
def estres(signals):
    # Estimate the number of steps as the range divided by the minimum increment. 
    
    nsig = np.size(signals)[1]
    # The estimated resolution in bits rounded up
    res = np.zeros(nsig)
    
    for ch in range(0, nsig):
        sortedsig = np.sort(signals[:,ch])
        min_inc = min(np.diff(sortedsig))
        
        if min_inc == 0:
            # Case where signal is flat. Resolution is 0.  
            continue
        else:
            nlevels = 1 + (sortedsig[-1]-sortedsig[0])/min_inc
            if nlevels>=reslevels[-1]:
                res[ch] = 32
            else:
                res[ch] = np.where(reslevels>nlevels)[0][0]
            
    return res
    

# Return the most suitable wfdb format to use given a signal resolution. Limited in output options. 
def wfdbfmt(res):
    if res<=8:
        return '80'
    elif res<=12:
        return '212'
    elif res<=16:
        return '16'
    elif res<=24:
        return '24'
    else:
        return '32'

# Return the resolution of the WFDB format. Returns a list if the input has multiple items. 
def wfdbfmtres(fmt):
    
    if type(fmt) in [list, np.ndarray]:
        numel = np.shape(fmt)[0]
        res = np.zeros(numel)
        for i in range(0, numel):
            res[i] = wfdbfmtres(fmt[i])
        return res
    
    if fmt in ['8', '80']:
        return 8
    elif fmt in ['310', '311']:
        return 10
    elif fmt == '212':
        return 12
    elif fmt in ['16', '61']:
        return 16
    elif fmt = '24':
        return 24
    elif fmt = '32':
        return 32
    else:
        sys.exit('Invalid WFDB format.')
    
# Calculate the checksums of a multi-channel signal
def calc_checksum(signals):
    return np.sum(signals, 0) % 65536

# Record specification fields            
recfieldspecs = OrderedDict([('recordname', WFDBfield([[str], '', None, True, 0])),
                         ('nseg', WFDBfield([[int], '/', 'recordname', True, 0])), # Essential for multi but not present in single.
                         ('nsig', WFDBfield([[int], ' ', 'recordname', True, 1])),
                         ('fs', WFDBfield([[int, float], ' ', 'nsig', True, 0])),
                         ('counterfreq', WFDBfield([[int, float], '/', 'fs', False, 0])),
                         ('basecounter', WFDBfield([[int, float], '(', 'counterfreq', False, 0])),
                         ('siglen', WFDBfield([[int], ' ', 'fs', True, 1])),
                         ('basetime', WFDBfield([[str], ' ', 'siglen', False, 2])),
                         ('basedate', WFDBfield([[str], ' ', 'basetime', False, 0]))])
# Signal specification fields 
sigfieldspecs = OrderedDict([('filename', WFDBfield([[str], '', None, True, 2])),
                         ('fmt', WFDBfield([[int, str], ' ', 'filename', True, 2])),
                         ('sampsperframe', WFDBfield([[int], 'x', 'fmt', False, 0])),
                         ('skew', WFDBfield([[int], ':', 'fmt', False, 0])),
                         ('byteoffset', WFDBfield([[int], '+', 'fmt', False, 0])),
                         ('adcgain', WFDBfield([[int], ' ', 'fmt', True, 1])),
                         ('baseline', WFDBfield([[int], '(', 'adcgain', True, 1])),
                         ('units', WFDBfield([[str], '/', 'adcgain', True, 2])),
                         ('adcres', WFDBfield([[int], ' ', 'adcgain', False, 2])),
                         ('adczero', WFDBfield([[int], ' ', 'adcres', False, 2])),
                         ('initvalue', WFDBfield([[int], ' ', 'adczero', False, 1])),
                         ('checksum', WFDBfield([[int], ' ', 'initvalue', False, 1])),
                         ('blocksize', WFDBfield([[int], ' ', 'checksum', False, 2])),
                         ('signame', WFDBfield([[str], ' ', 'blocksize', False, 1]))])
    
# Segment specification fields
segfieldspecs = OrderedDict([('segname', WFDBfield([[str], '', None, True, 0])),
                         ('seglen', WFDBfield([[int], ' ', 'segname', True, 0]))])




IndentationError: expected an indented block (<ipython-input-2-39f8aa9a3ede>, line 50)

In [48]:
-102 % 100

98